# Geopolygon Extraction From Studio Using SQL

### This Jupyter Notebook extracts geopolgyons from Studio. The extraction only exports geopolygon outlines and not the attribute information associated with each polygon. Basic geopolygon information like the name, guid, and studio folder path is provided. 

In [302]:
###IMPORT  LIBRARIES#####
import pandas as pd
import arcgis
from arcgis.features import GeoAccessor,GeoSeriesAccessor
from datetime import datetime
import sqlalchemy
from sqlalchemy import create_engine
import cx_Oracle
import os
import arcpy
startTime = datetime.now()

###connect to studio###
engine = create_engine('oracle://@P_PET_SAMP')
connection = engine.connect()
print(engine) 

Engine(oracle://@P_PET_SAMP)


In [303]:
#Setting Variables
arcpy.env.overwriteOutput = True
Studio_SDE = "I:\\UTCS\\Region\\SAmerica.Hou\\Ref_Data\\Guyana.hou\\General\\Maps\\Guyana_INT_to_Guyana_SDE\\HA_P_PET_SAMP.sde"
Project_gdb = "C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\MyProject.gdb"
splittatt = "C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\splittatt.gdb"

#### The query cell is a sql query that can be modified to whatever is needed.

In [304]:
geopolygon_query = ''' SELECT 
                dbxobj.name NAME,
                dbxobj.folder_path FOLDER_PATH,
                dbxobj.display_type Display_Type,
                dbxobj.domain_object Domain_Object,
                dbxobj.guid GUID,
                dbxobj.shape SHAPE,
                sdo_util.getnumelem(dbxobj.shape) NumElements,
                sdo_util.getnumvertices(dbxobj.shape) NumVertices

            FROM 
                GUYANA_INT.DBX_OBJECT dbxobj

            WHERE 
                GENERIC_DISPLAY_TYPE = 'Geopolygons'  '''


In [305]:
#wanted to see the query in dataframe view. Going to use this to join attributes later.
polygons = pd.read_sql_query(sql = geopolygon_query,con=connection)
polygons.columns = polygons.columns.str.upper()
polygons = polygons.drop(columns=["SHAPE"])
display(polygons.head())

#creating query layer from geopolygon query. Query layer by itself does not render, but once exported to feature, it renders fine. The polygons from studio are stored as polylines. Even the API has polygons entityeType as DbxPolylinesSet
Geopolygon_layer = arcpy.MakeQueryLayer_management(input_database= Studio_SDE, out_layer_name="Geopolygon_layer", query=geopolygon_query, oid_fields=["GUID"], shape_type="POLYLINE", srid="4326")
#creating feature class from query layer
Geopolygon_feature = arcpy.FeatureClassToFeatureClass_conversion(in_features= Geopolygon_layer, out_path= Project_gdb, out_name="Geopolygons_from_Studio")

,NAME,FOLDER_PATH,DISPLAY_TYPE,DOMAIN_OBJECT,GUID,NUMELEMENTS,NUMVERTICES
0,GUYANA_AOI_Filter,\Project\Input\Reference Datasets\Regional,Geopolygons,DbxGeopolygonSet,://+1__GeopolygonModule.Persistence.Geopolygon/+1__6c6964e8-3b0a-4142-a8d9-81e6a522d33f,1,12451
1,Sorubim_3D,\Project\Input\Reference Datasets\Polygons,Geopolygons,DbxGeopolygonSet,://+1__GeopolygonModule.Persistence.Geopolygon/+1__8f2db58f-303f-448c-a826-69a06847cc18,1,11
2,Guyana_3d_17k,\Project\Input\Reference Datasets\Polygons,Geopolygons,DbxGeopolygonSet,://+1__GeopolygonModule.Persistence.Geopolygon/+1__4cb9089a-98dc-4519-9496-3dce33c93193,1,33
3,Guyana_3D_17k_Sorubim_3D_Merge,\Project\Input\Reference Datasets\Polygons,Geopolygons,DbxGeopolygonSet,://+1__GeopolygonModule.Persistence.Geopolygon/+1__8711e036-6e32-408f-b9ee-f47fab284c6e,2,45
4,Payara_Prod_License_Area,\Project\Input\Reference Datasets\Polygons,Geopolygons,DbxGeopolygonSet,://+1__GeopolygonModule.Persistence.Geopolygon/+1__957797c5-8768-4e05-b3aa-56fd071200ec,3,19


In [306]:
#Studio outputs polyline feature of all geopolygons. If you don't need them merged, you can run the splitbyattributes and will create separate features based on a certain attribute.
fields = ["NAME"]
arcpy.analysis.SplitByAttributes(Geopolygon_feature,splittatt,fields)

<Result 'C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\splittatt.gdb'>

In [307]:
#If you need a merged geopolygon set, but as polygons, this cell converts each record to polygon, adds back the GUID attribute and merges all records again. 
#Final product is a polygon feature class with all the polygons. When converting from Polyline to Polygon, each element is a separate record.
arcpy.env.workspace = splittatt
arcpy.env.overwriteOutput = True
fields = ["SHAPE@","NAME","FOLDER_PATH","GUID","DISPLAY_TYPE","NUMELEMENTS","NUMVERTICES"]
with arcpy.da.SearchCursor(Geopolygon_feature,fields) as cursor:
    for row in cursor:
        rowreplace = row[1].replace(" ","_")
        rowreplace = rowreplace.replace("-","")
        print(row)
        print(row[0])
        print(rowreplace)
        feature = arcpy.management.CopyFeatures(row[0],f"fc_{rowreplace}")
        feat2poly = arcpy.management.FeatureToPolygon(feature,f"{feature}_poly")
        field = "GUID"
        expression = row[3]
        print(expression)
        arcpy.AddField_management(feat2poly,field,"TEXT")
        arcpy.CalculateField_management(feat2poly,field,'"'+expression+'"',"PYTHON3")

merge_list = arcpy.ListFeatureClasses("*_poly")
Geopolygon_studio = arcpy.management.Merge(merge_list,f"Geopolygons_studio_GUID")

(<Polyline object at 0x1cac81ce9b0[0x1cb6d12ea80]>, 'GUYANA_AOI_Filter', '\\Project\\Input\\Reference Datasets\\Regional', '://+1__GeopolygonModule.Persistence.Geopolygon/+1__6c6964e8-3b0a-4142-a8d9-81e6a522d33f', 'Geopolygons', 1.0, 12451.0)
<geoprocessing describe geometry object object at 0x000001CB6D12EA80>
GUYANA_AOI_Filter
://+1__GeopolygonModule.Persistence.Geopolygon/+1__6c6964e8-3b0a-4142-a8d9-81e6a522d33f
(<Polyline object at 0x1cb6d0f9cc0[0x1cb6d12e8a0]>, 'Sorubim_3D', '\\Project\\Input\\Reference Datasets\\Polygons', '://+1__GeopolygonModule.Persistence.Geopolygon/+1__8f2db58f-303f-448c-a826-69a06847cc18', 'Geopolygons', 1.0, 11.0)
<geoprocessing describe geometry object object at 0x000001CB6D12E8A0>
Sorubim_3D
://+1__GeopolygonModule.Persistence.Geopolygon/+1__8f2db58f-303f-448c-a826-69a06847cc18
(<Polyline object at 0x1cac81ce9b0[0x1cb6d12ea80]>, 'Guyana_3d_17k', '\\Project\\Input\\Reference Datasets\\Polygons', '://+1__GeopolygonModule.Persistence.Geopolygon/+1__4cb9089a

In [308]:
#creating spatial dataframe from the outputted feature class. Going to merge the attributes from the polygons dataframe with the spatial dataframe and create a new final feature class.
sdf = pd.DataFrame.spatial.from_featureclass(Geopolygon_studio)

final = pd.merge(sdf,polygons,how="left",left_on=["GUID"],right_on=["GUID"])
print("Creating Geopolygon feature...")
final.spatial.to_featureclass(location="C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\splittatt.gdb\\Geopolygons_final",overwrite=True) #this cell will fail if feature is open in project.
finaltime = datetime.now()-startTime
print(f"Total time to run: {finaltime}")

Creating Geopolygon feature...
Total time to run: 0:02:15.647872
